In [207]:
import os
import sys
import openpyxl
import pygal
import pandas as pd
import numpy as np
from pandas import ExcelWriter
from pandas import ExcelFile
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.graphics import mosaicplot
from statsmodels.graphics.mosaicplot import mosaic
sns.set(style="ticks", color_codes=True)
from rpy2.robjects.packages import importr
import requests
import xlrd
import warnings
warnings.filterwarnings("ignore")
plt.style.use('ggplot')

In [208]:
#  Doctorate Recipients by DemographDoctrate reciepients by Citizenship,  Race/Ethnicity, Sex, S&E Fields, Broad Fields, Detailed Fields, and Year
# Data Download from NCSES Interactive Data Tool
df1 = pd.read_csv('C:/Users/Muluken/WorkingFiles/NSF_NCSES/Data/SED_Export_2020-06-17T23_22_18.852Z.csv', skiprows=[0], header=[1])
df1 = df1.set_index(['S&E Fields', 'Broad Fields', 'Detailed Fields'])
df1.head(3)
#df1.tail()
df1.dtypes

2018    object
2017     int64
2016    object
2015    object
2014    object
         ...  
1962    object
1961    object
1960    object
1959    object
1958    object
Length: 61, dtype: object

In [209]:
df1s = df1.stack()
df1s =df1s.to_frame()
df1s.head(10)
df1s = df1s.reset_index()
df1s.columns = ['S&E Fields', 'Broad Fields', 'Detailed Fields', 'Year', 'Number']
# df1s.columns
# df1s.head()
# df1s.shape

In [210]:
df1s = df1s[df1s['Detailed Fields'] != 'Total']
df1s = df1s.reset_index()
df1s = df1s[['S&E Fields', 'Broad Fields', 'Detailed Fields', 'Year', 'Number']]
#df1s["Number"] = df1s["Number"].astype(str).astype(float)
df1s.head(3)
df1s.tail(3)
df1s['Number'].replace('-', np.nan, inplace=True)
df1s = df1s.dropna()
#df1s.describe
#df1s.dtypes

df1s.describe
df1s["Number"] = df1s["Number"].astype(str).astype(int)
df1s.dtypes
df1s.head(3)
#df1s.tail(3)

,S&E Fields,Broad Fields,Detailed Fields,Year,Number
0,Science and engineering,Life sciences,Agricultural sciences and natural resources,2018,1445
1,Science and engineering,Life sciences,Agricultural sciences and natural resources,2017,1494
2,Science and engineering,Life sciences,Agricultural sciences and natural resources,2016,1378


In [211]:
SE_NSE = df1s[[ 'S&E Fields','Broad Fields','Detailed Fields','Year','Number']]
SE_NSE = df1s.set_index(['S&E Fields', 'Broad Fields', 'Detailed Fields'])

In [217]:
SE_NSE = SE_NSE.groupby(['S&E Fields','Broad Fields', 'Detailed Fields'])['Number'].sum()
#SE_NSE=SE_NSE.sum(level='Detailed Fields')
SE_NSE = SE_NSE.to_frame()
SE_NSE = SE_NSE.reset_index()

In [218]:
SE_NSE.head(200)
#SE_NSE.columns

,S&E Fields,Broad Fields,Detailed Fields,Number
0,Non-science and engineering,Education,Education administration,89994
1,Non-science and engineering,Education,Education research,141889
2,Non-science and engineering,Education,Other education,33149
3,Non-science and engineering,Education,Teacher education,22151
4,Non-science and engineering,Education,Teaching fields,58215
5,Non-science and engineering,Humanities and arts,Foreign languages and literature,33948
6,Non-science and engineering,Humanities and arts,History,51058
7,Non-science and engineering,Humanities and arts,Letters,77297
8,Non-science and engineering,Humanities and arts,Other humanities and arts,94103
9,Non-science and engineering,Other non-S&E,Business management and administration,56686


In [179]:
# Doctorate Recipients by Demographic Characteristics ---- Data by Citizenship,  Race/Ethnicity, Sex, S&E Fields, Broad Fields, Detailed Fields, and Year
# Data Download from NCSES Interactive Data Tool
df2 = pd.read_csv('C:/Users/Muluken/WorkingFiles/NSF_NCSES/Data/SED_Export_2020-06-17T23_29_53.896Z.csv', skiprows=[0], header=[1])
df2 = df2.set_index(['Citizenship','Race and Ethnicity', 'Sex','S&E Fields', 'Broad Fields', 'Detailed Fields'])
df2.head()


2018  \
Citizenship                        Race and Ethnicity Sex   S&E Fields              Broad Fields  Detailed Fields          
U.S. citizen or permanent resident Total              Total Total                   Total         Total            35404   
                                   Hispanic or Latino Total Total                   Total         Total             2582   
                                                      Male  Total                   Total         Total             1202   
                                                            Science and engineering Total         Total              908   
                                                                                    Life sciences Total              267   

                                                                                                                    2017  \
Citizenship                        Race and Ethnicity Sex   S&E Fields              Broad Fields  Detailed Fields          
U.S. citizen or permanent resident Total              Total Total                   Total         Total            35738   
                                   Hispanic or Latino Total Total                   Total         Total             2537   
                                                      Male  Total                   Total         Total             1153   
                                                            Science and engineering Total         Total              845   
                                                                                    Life sciences Total              242   

                                                                                                                    2016  \
Citizenship                        Race and Ethnicity Sex   S&E Fields              Broad Fields  Detailed Fields          
U.S. citizen or permanent resident Total              Total Total                   Total         Total            35671   
                                   Hispanic or Latino Total Total                   Total         Total             2548   
                                                      Male  Total                   Total         Total             1134   
                                                            Science and engineering Total         Total              849   
                                                                                    Life sciences Total              242   

                                                                                                                    2015  \
Citizenship                        Race and Ethnicity Sex   S&E Fields              Broad Fields  Detailed Fields          
U.S. citizen or permanent resident Total              Total Total                   Total         Total            35071   
                                   Hispanic or Latino Total Total                   Total         Total             2448   
                                                      Male  Total                   Total         Total             1119   
                                                            Science and engineering Total         Total              832   
                                                                                    Life sciences Total              273   

                                                                                                                    2014  \
Citizenship                        Race and Ethnicity Sex   S&E Fields              Broad Fields  Detailed Fields          
U.S. citizen or permanent resident Total              Total Total                   Total         Total            34004   
                                   Hispanic or Latino Total Total                   Total         Total             2190   
                                                      Male  Total                   Total         Total              999   
                    

In [ ]:
df2s = df2.stack()
df2s =df2s.to_frame()

df2s = df2s.reset_index()
df2s.columns = ['Citizenship','Race and Ethnicity', 'Sex','S&E Fields', 'Broad Fields', 'Detailed Fields', 'Year', 'Number']
df2s.head(10)

In [ ]:
df2s = df2s[df2s['Detailed Fields'] != 'Total']
df2s = df2s.reset_index()
df2s = df2s[['Citizenship','Race and Ethnicity', 'Sex','S&E Fields', 'Broad Fields', 'Detailed Fields', 'Year', 'Number']]
df2s.head()